<a href="https://colab.research.google.com/github/tlavette/ARepositoryName/blob/master/ieeefraud_kaggleAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install kaggle

In [1]:
!pip install autogluon


In [2]:
# copy kaggle.json to /root/.kaggle/ folder so that kaggle cli can access it.
!mkdir /.kaggle
!mv kaggle.json /.kaggle
!mv /.kaggle /root/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c ieee-fraud-detection


 98% 116M/118M [00:06<00:00, 25.4MB/s]
100% 118M/118M [00:06<00:00, 18.4MB/s]


In [4]:
!unzip ieee-fraud-detection.zip

Archive:  ieee-fraud-detection.zip
  inflating: sample_submission.csv   
  inflating: test_identity.csv       
  inflating: test_transaction.csv    
  inflating: train_identity.csv      
  inflating: train_transaction.csv   


**Beginning of File Management. Read Train CSV files**

In [5]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = '/content/'  # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'AutoGluonModels/'  # where to store trained models

train_identity = pd.read_csv(directory+'train_identity.csv')
train_transaction = pd.read_csv(directory+'train_transaction.csv')


**Merge both CSV files - Observe "isFraud"**

In [6]:
train_data= pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
train_data_sampled = train_data.sample(frac=0.01, random_state=42)  # Use 1% of the data

print(train_data_sampled.head())





        TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  \
470624        3457624        0       12153579         724.000         W   
565820        3552820        0       15005886         108.500         W   
284083        3271083        0        6970178          47.950         W   
239689        3226689        0        5673658         100.599         C   
281855        3268855        0        6886780         107.950         W   

        card1  card2  card3       card4  card5  ... id_31  id_32  id_33  \
470624   7826  481.0  150.0  mastercard  224.0  ...   NaN    NaN    NaN   
565820  12544  321.0  150.0        visa  226.0  ...   NaN    NaN    NaN   
284083   9400  111.0  150.0  mastercard  224.0  ...   NaN    NaN    NaN   
239689  15885  545.0  185.0        visa  138.0  ...   NaN    NaN    NaN   
281855  15497  490.0  150.0        visa  226.0  ...   NaN    NaN    NaN   

        id_34  id_35 id_36 id_37  id_38  DeviceType  DeviceInfo  
470624    NaN    NaN   NaN   NaN

**Define predictor of the newly merged Data file**

In [7]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(train_data_sampled, presets='best_quality', dynamic_stacking=False, num_stack_levels=1, time_limit=360
)

results = predictor.fit_summary()


Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          8
GPU Count:          1
Memory Avail:       43.73 GB / 50.99 GB (85.8%)
Disk Space Avail:   188.37 GB / 235.68 GB (79.9%)
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True, 'num_bag_sets': 1, 'num_stack_levels': 1}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_ray_logging': True,
             'holdout_data': None,
             'holdout_frac': 0.1111111111111111,
             'memory_safe_fits': True,
     

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   0.888999     roc_auc       7.665932  233.328618                0.001667           0.480841            3       True         21
1       WeightedEnsemble_L2   0.883612     roc_auc       4.132498  191.977718                0.001212           0.297450            2       True         13
2         LightGBMXT_BAG_L2   0.877450     roc_auc       6.932003  219.944287                0.241752           9.954139            2       True         14
3            XGBoost_BAG_L1   0.876396     roc_auc       0.330295   15.323561                0.330295          15.323561            1       True         10
4           CatBoost_BAG_L2   0.873986     roc_auc       7.135750  275.547814                0.445500          65.557666            2       True         18
5 

**Use trained AutoGluon Predictor to make predictions on the TEST DATA.
Merge both CSV files **bold text**

In [9]:
!pip install autogluon.tabular

import pandas as pd
from autogluon.tabular import TabularPredictor


test_identity=pd.read_csv(directory+'test_identity.csv')
test_transaction=pd.read_csv(directory+'test_transaction.csv')

# Ensure all columns from both test datasets are included in test_data
test_data=pd.merge(test_transaction,test_identity,on='TransactionID',how='outer')

# Check for missing columns in test_data compared to training data
missing_cols = set(train_data_sampled.columns) - set(test_data.columns)
if missing_cols:
    print(f"Missing columns in test_data: {missing_cols}")
    # Handle missing columns (e.g., add them with default values)
    for col in missing_cols:
        test_data[col] = 0  # Example: fill missing columns with 0

y_predproba=predictor.predict_proba(test_data)
y_predproba.head(5)

Missing columns in test_data: {'id_25', 'id_05', 'isFraud', 'id_20', 'id_37', 'id_03', 'id_23', 'id_16', 'id_31', 'id_35', 'id_24', 'id_13', 'id_09', 'id_01', 'id_17', 'id_27', 'id_07', 'id_12', 'id_30', 'id_38', 'id_15', 'id_28', 'id_18', 'id_36', 'id_19', 'id_04', 'id_11', 'id_22', 'id_14', 'id_21', 'id_32', 'id_34', 'id_02', 'id_06', 'id_29', 'id_33', 'id_08', 'id_10', 'id_26'}


Loading: /content/AutoGluonModels/models/CatBoost_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/ExtraTreesEntr_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/KNeighborsDist_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/LightGBMLarge_BAG_L1/model.pkl
/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
Loading: /content/AutoGluonModels/models/LightGBMXT_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/LightGBM_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/

,0,1
0,0.982677,0.017323
1,0.984488,0.015512
2,0.956906,0.043093
3,0.986915,0.013085
4,0.979347,0.020653


**Moving on to Predictor Steps and Submission**

In [10]:
predictor.positive_class

1

In [11]:
predictor.class_labels # classes in this list correspond to columns of predict_proba() output

[0, 1]

In [14]:
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)

Loading: /content/AutoGluonModels/models/CatBoost_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/ExtraTreesEntr_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/ExtraTreesGini_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/KNeighborsDist_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/LightGBMLarge_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/LightGBMXT_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/LightGBM_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/RandomForestEntr_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/RandomForestGini_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/XGBoost_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/ExtraTreesGini_BAG_L2/model.pkl
Loading: /content/AutoGluonModels/models/LightGBMXT_BAG_L2/model.pkl
Lo

In [15]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)

In [17]:
!kaggle competitions submit -c ieee-fraud-detection -f sample_submission.csv -m "ieee submission. My second submission"

100% 5.80M/5.80M [00:02<00:00, 2.10MB/s]
Successfully submitted to IEEE-CIS Fraud Detection

**END**